In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from scipy.stats import norm
from causal_nets import causal_net_estimate
from joblib import Parallel, delayed, cpu_count
from savReaderWriter import SavReaderNp
import random
from more_itertools import sliced
from scipy import stats
import matplotlib.mlab as mlab

Code to replicate the simulations in the Master's Thesis Neural Nets for Treatment Effect Estimation by Hugo Foerster-Baldenius.
Works by including packages and loops and then each simulation can be run seperately. Relies heavily on the causal nets package by Milica Popovic that is a companion to (Farrell, Liang, Misra 2019). The cross fitting simulation is available in a separate file. 
# Simulation 1
## Linear DGP (replication from paper)

In [110]:
##functions for simulation 1
def loop_inside(number, ATT, N, runs, sample, model, nconsumer_characteristics, hidden_layer_sizes, estimate_ps, alpha_p, alpha_mu, alpha_tau,dropout_rates):
        seed = random.randint(1, 100000)
        np.random.seed(seed)
        X = np.hstack((np.repeat(1,N).reshape(N,1), np.random.uniform(low=0, high=1, size=[N, nconsumer_characteristics])))
        if sample=='randomized':
            prob_of_T = 0.5
        else:
            p_of_t = np.dot(X[:, :21], alpha_p)
            p_of_t = np.reshape(p_of_t, -1)
            p_of_t = p_of_t.reshape(-1)
            prob_of_T = 1/(1+np.exp(-p_of_t))


        T = np.random.binomial(size=N, n=1, p=prob_of_T)

        epsilon= np.random.normal(0, 1, N)

        mu0_real= np.dot(X, alpha_mu.T) 
        tau_real = np.dot(X, alpha_tau) 

        if model == 'quadratic':
                count = comb(nconsumer_characteristics, 2, True, True)
                beta_tau = np.random.uniform(low=-0.05, high=0.06, size=count)
                tau_real = tau_real + _sum_polynomial_X_times_weights(X, beta_tau)
                beta_mu0 = np.random.normal(loc=0.01, scale=0.3, size=count)
                mu0_real = mu0_real +_sum_polynomial_X_times_weights(X, beta_mu0)
        else:
                beta_tau = None
                beta_mu = None

        if estimate_ps == False:
            hidden_layer_sizes_t= None 
        else:
            hidden_layer_sizes_t=[30]
            

        Y = mu0_real + tau_real*T + epsilon
        
        X_train, X_valid, T_train, T_valid, Y_train, Y_valid = train_test_split(X, T, Y, test_size=0.1, random_state=42)
        
        tau_pred, mu0_pred, prob_t_pred, psi_0, psi_1, history, history_ps = causal_net_estimate(
        [X_train, T_train, Y_train], [X_valid, T_valid, Y_valid], [X, T, Y], hidden_layer_sizes, hidden_layer_sizes_t=hidden_layer_sizes_t,
        dropout_rates=dropout_rates, batch_size=None, alpha=0., r_par=0., optimizer='Adam', learning_rate=0.0009,
        max_epochs_without_change=30, max_nepochs=10000, seed=None, estimate_ps=estimate_ps, verbose=False)
       
        #from the characteristics of the uniform distribution on [0,1] the expected value is .5
        tau_true_mean = np.sum(0.5 * alpha_tau) + .5*alpha_tau[0]
        part_one_var=np.sum((1/3)*np.power(alpha_tau,2))+(2/3)*np.power(alpha_tau[0],2)
        part_two_var=np.sum((2/9)*np.dot(alpha_tau,alpha_mu))+(7/9)*alpha_tau[0]*alpha_mu[0]
        part_three_var=np.sum((1/3)*np.power(alpha_mu,2))+(2/3)*np.power(alpha_mu[0],2)
        tau_var= part_one_var+part_two_var+part_three_var -np.power(tau_true_mean,2)
        
        if ATT == False:
            # Calculate the average treatment effect
            ate = np.mean(psi_1-psi_0)
            mse = np.power((tau_true_mean-ate),2)
            # Calculate the bias
            # Calculate the 95% confidence interval for average treatment effect
            CI_lowerbound = ate - norm.ppf(0.975)*np.std(psi_1-psi_0)/np.sqrt(len(psi_0))
            CI_upperbound = ate + norm.ppf(0.975)*np.std(psi_1-psi_0)/np.sqrt(len(psi_0))

            IL=CI_upperbound-CI_lowerbound

            # How to do coverage??
            if tau_true_mean>CI_lowerbound and tau_true_mean<CI_upperbound:
                coverage = 1 
            else:
                coverage = 0

            return mse, ate, IL, coverage, tau_true_mean, tau_var
    
        if ATT == True:
        
            part_one_one= (T*(mu0_pred+ tau_pred))/np.mean(T)
            part_one_zero= (T*(mu0_pred))/np.mean(T)
            part_two_one= ((prob_t_pred)/np.mean(T))*(T*((Y-mu0_pred-tau_pred)/prob_t_pred))
            part_two_zero= ((prob_t_pred)/np.mean(T))*((1-T)*((Y-mu0_pred)/(1-prob_t_pred)))                                                          

            psi_one=part_one_one+part_two_one
            psi_zero=part_one_zero+part_two_zero
            att = np.mean(psi_one-psi_zero)
            mse = np.power((tau_true_mean-att),2)
            
            # Calculate the 95% confidence interval for average treatment effect
            CI_lowerbound = att - norm.ppf(0.975)*np.std(psi_one-psi_zero)/np.sqrt(len(psi_one))
            CI_upperbound = att + norm.ppf(0.975)*np.std(psi_one-psi_zero)/np.sqrt(len(psi_zero))
        
            IL=CI_upperbound-CI_lowerbound
        
            # How to do coverage??
            if tau_true_mean>CI_lowerbound and tau_true_mean<CI_upperbound:
                coverage = 1 
            else:
                coverage = 0
            
            return mse, att, IL, coverage, tau_true_mean, tau_var
        
def simulator(ATT, N, runs, sample, model, nconsumer_characteristics, hidden_layer_sizes, estimate_ps, dropout_rates=None):
    coverage=[]
    mse=[]
    IL=[]
    ##to always use same DGP
    np.random.seed(42)
    alpha_p=np.append(0.09, np.random.uniform(-.55,.55,size=[nconsumer_characteristics,1]))
    alpha_mu = np.append(.09, np.random.normal(loc=0.3, scale=0.7, size=[1, nconsumer_characteristics]))
    alpha_tau=np.append(-0.05,np.random.uniform(0.1,0.22,20))

    r = Parallel(n_jobs=4)(delayed(loop_inside)(number, ATT, N, runs, sample, model, nconsumer_characteristics, hidden_layer_sizes, estimate_ps, alpha_p, alpha_mu, alpha_tau, dropout_rates) for number in range(1, runs+1))
    mse, ate, IL, coverage, tau_true_mean, tau_var = zip(*r)     
    return mse, ate, IL, coverage, tau_true_mean,tau_var



def calculate_statistics(se, ate_real, ate, IL, coverage):
    mse= np.mean(se)
    bias=np.mean(ate)-ate_real[1]
    IL=np.mean(IL)
    coverage=np.mean(coverage)
    return(mse,bias,IL,coverage)

In [111]:
se1, ate1, IL1, coverage1, tau_true_mean1, tau_var1 =simulator(False, N=1000, runs=5, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,5],estimate_ps=False,dropout_rates=None)

In [133]:
se1, ate1, IL1, coverage1, tau_true_mean1 =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,5],estimate_ps=False,dropout_rates=None)

In [134]:
se2, ate2, IL2, coverage2, tau_true_mean2 =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20],estimate_ps=False,dropout_rates=None)

In [135]:
se3, ate3, IL3, coverage3, tau_true_mean3  =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[80,80,80],estimate_ps=False, dropout_rates=None)

In [136]:
se4, ate4, IL4, coverage4, tau_true_mean4  =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10,5],estimate_ps=False, dropout_rates=None)

In [137]:
se5, ate5, IL5, coverage5, tau_true_mean5 =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20,10],estimate_ps=False, dropout_rates=None)

In [138]:
mse1,bias1,IL1,coverage1=calculate_statistics(se1, tau_true_mean1, ate1, IL1, coverage1)
mse2,bias2,IL2,coverage2=calculate_statistics(se2, tau_true_mean2, ate2, IL2, coverage2)
mse3,bias3,IL3,coverage3=calculate_statistics(se3, tau_true_mean3, ate3, IL3, coverage3)
mse4,bias4,IL4,coverage4=calculate_statistics(se4, tau_true_mean4, ate4, IL4, coverage4)
mse5,bias5,IL5,coverage5=calculate_statistics(se5, tau_true_mean5, ate5, IL5, coverage5)

In [139]:
print(mse1, bias1, IL1, coverage1)
print(mse2, bias2, IL2, coverage2)
print(mse3, bias3, IL3, coverage3)
print(mse4, bias4, IL4, coverage4)
print(mse5, bias5, IL5, coverage5)

0.005304423990536955 -0.0068608161035013104 0.25021645061144276 0.928
0.004380687926552355 -0.0027091905570939545 0.24425839355676676 0.936
0.004307933149167905 -0.0011866314982622583 0.24226490971036152 0.928
0.005912006123682452 -0.013181624641499523 0.24931652039923666 0.897
0.004797684353514436 -0.00818128446232902 0.24420248462707403 0.928


# Simulation 2
## Estimating ATT with propensity score

In [142]:
se2_1, ate2_1, IL2_1, coverage2_1, tau_true_mean2_1 =simulator(True, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,5],estimate_ps=True)

In [143]:
se2_2, ate2_2, IL2_2, coverage2_2, tau_true_mean2_2 =simulator(True, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20],estimate_ps=True)

In [144]:
se2_3, ate2_3, IL2_3, coverage2_3, tau_true_mean2_3 =simulator(True, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[80,80,80], estimate_ps=True)

In [145]:
se2_4, ate2_4, IL2_4, coverage2_4, tau_true_mean2_4 =simulator(True, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10,5], estimate_ps=True)

In [146]:
se2_5, ate2_5, IL2_5, coverage2_5, tau_true_mean2_5 =simulator(True, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20,10], estimate_ps=True)

In [196]:
mse2_1,bias2_1,IL2_1,coverage2_1=calculate_statistics(se2_1, tau_true_mean2_1, ate2_1, IL2_1, coverage2_1)
mse2_2,bias2_2,IL2_2,coverage2_2=calculate_statistics(se2_2, tau_true_mean2_2, ate2_2, IL2_2, coverage2_2)
mse2_3,bias2_3,IL2_3,coverage2_3=calculate_statistics(se2_3, tau_true_mean2_3, ate2_3, IL2_3, coverage2_3)
mse2_4,bias2_4,IL2_4,coverage2_4=calculate_statistics(se2_4, tau_true_mean2_4, ate2_4, IL2_4, coverage2_4)
mse2_5,bias2_5,IL2_5,coverage2_5=calculate_statistics(se2_5, tau_true_mean2_5, ate2_5, IL2_5, coverage2_5)

In [197]:
print("%.5f" % mse2_1, "%.5f" % bias2_1, "%.5f" % IL2_1, "%.3f" % coverage2_1)
print("%.5f" % mse2_2, "%.5f" % bias2_2, "%.5f" % IL2_2, "%.3f" % coverage2_2)
print("%.5f" % mse2_3, "%.5f" % bias2_3, "%.5f" % IL2_3, "%.3f" % coverage2_3)
print("%.5f" % mse2_4, "%.5f" % bias2_4, "%.5f" % IL2_4, "%.3f" % coverage2_4)
print("%.5f" % mse2_5, "%.5f" % bias2_5, "%.5f" % IL2_5, "%.3f" % coverage2_5)

0.00853 0.00332 0.41965 0.977
0.01017 0.00197 0.42953 0.962
0.00952 -0.00174 0.42975 0.973
0.00947 -0.00189 0.41829 0.962
0.00886 -0.00139 0.42425 0.975


# Simulation 3
## Implementing Dropout
### Architecture 1

In [149]:
##Dropout 20 percent
se1_drop, ate1_drop, IL1_drop, coverage1_drop, tautruemean1_drop =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,5],estimate_ps=False,dropout_rates=[.20,.20,.20])

In [150]:
##Dropout 40 percent
se1_drop2, ate1_drop2, IL1_drop2, coverage1_drop2, tautruemean1_drop2 =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,5],estimate_ps=False,dropout_rates=[.40,.40,.40])

In [151]:
##Droupout 60 percent
se1_drop3, ate1_drop3, IL1_drop3, coverage1_drop3, tautruemean1_drop3 =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,5],estimate_ps=False,dropout_rates=[.60,.60,.60])

In [152]:
mse1_drop, bias1_drop, IL1_drop, coverage1_drop=calculate_statistics(se1_drop, tautruemean1_drop, ate1_drop, IL1_drop, coverage1_drop)
mse1_drop2, bias1_drop2, IL1_drop2, coverage1_drop2=calculate_statistics(se1_drop2, tautruemean1_drop2, ate1_drop2, IL1_drop2, coverage1_drop2)
mse1_drop3, bias1_drop3, IL1_drop3, coverage1_drop3=calculate_statistics(se1_drop3, tautruemean1_drop3, ate1_drop3, IL1_drop3, coverage1_drop3)

In [194]:
print("%.5f" % mse1_drop,"%.5f" %  bias1_drop, "%.5f" %  IL1_drop,"%.3f" %  coverage1_drop)
print("%.5f" % mse1_drop2,"%.5f" %  bias1_drop2, "%.5f" %  IL1_drop2,"%.3f" %  coverage1_drop2)
print("%.5f" % mse1_drop3,"%.5f" %  bias1_drop3, "%.5f" %  IL1_drop3,"%.3f" %  coverage1_drop3)

0.00579 -0.00890 0.28889 0.948
0.00552 -0.00155 0.30781 0.957
0.00624 -0.00167 0.35097 0.970


### Architecture 2

In [154]:
se2_drop, ate2_drop, IL2_drop, coverage2_drop, tautruemean2_drop =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20],estimate_ps=False,dropout_rates=[.20,.20,.20])

In [155]:
se2_drop2, ate2_drop2, IL2_drop2, coverage2_drop2, tautruemean2_drop2 =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20],estimate_ps=False,dropout_rates=[.40,.40,.40])

In [156]:
se2_drop3, ate2_drop3, IL2_drop3, coverage2_drop3, tautruemean2_drop3 =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20],estimate_ps=False,dropout_rates=[.60,.60,.60])

In [157]:
mse2_drop, bias2_drop, IL2_drop, coverage2_drop=calculate_statistics(se2_drop, tautruemean2_drop, ate2_drop, IL2_drop, coverage2_drop)
mse2_drop2, bias2_drop2, IL2_drop2, coverage2_drop2=calculate_statistics(se2_drop2, tautruemean2_drop2, ate2_drop2, IL1_drop2, coverage2_drop2)
mse2_drop3, bias2_drop3, IL2_drop3, coverage2_drop3=calculate_statistics(se2_drop3, tautruemean2_drop3, ate2_drop3, IL2_drop3, coverage2_drop3)

In [195]:
print("%.5f" % mse2_drop, "%.5f" %  bias2_drop, "%.5f" %  IL2_drop, "%.3f" %  coverage2_drop)
print("%.5f" % mse2_drop2, "%.5f" %  bias2_drop2, "%.5f" %  IL2_drop2, "%.3f" %  coverage2_drop2)
print("%.5f" % mse2_drop3, "%.5f" %  bias2_drop3, "%.5f" %  IL2_drop3, "%.3f" %  coverage2_drop3)

0.00464 -0.00434 0.25367 0.935
0.00509 -0.00554 0.30781 0.952
0.00546 -0.00435 0.28969 0.947


### Architecture 3

In [159]:
se3_drop, ate3_drop, IL3_drop, coverage3_drop, tautruemean3_drop =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[80,80,80],estimate_ps=False,dropout_rates=[.20,.20,.20])

In [160]:
se3_drop2, ate3_drop2, IL3_drop2, coverage3_drop2, tautruemean3_drop2 =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[80,80,80],estimate_ps=False,dropout_rates=[.40,.40,.40])

In [161]:
se3_drop3, ate3_drop3, IL3_drop3, coverage3_drop3, tautruemean3_drop3 =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[80,80,80],estimate_ps=False,dropout_rates=[.60,.60,.60])

In [162]:
mse3_drop, bias3_drop, IL3_drop, coverage3_drop=calculate_statistics(se3_drop, tautruemean3_drop, ate3_drop, IL3_drop, coverage3_drop)
mse3_drop2, bias3_drop2, IL3_drop2, coverage3_drop2=calculate_statistics(se3_drop2, tautruemean3_drop2, ate3_drop2, IL3_drop2, coverage3_drop2)
mse3_drop3, bias3_drop3, IL3_drop3, coverage3_drop3=calculate_statistics(se3_drop3, tautruemean3_drop3, ate3_drop3, IL3_drop3, coverage3_drop3)

In [199]:
print("%.5f" % mse3_drop, "%.5f" % bias3_drop, "%.5f" % IL3_drop, "%.3f" % coverage3_drop)
print("%.5f" % mse3_drop2, "%.5f" % bias3_drop2, "%.5f" % IL3_drop2, "%.3f" % coverage3_drop2)
print("%.5f" % mse3_drop3, "%.5f" % bias3_drop3, "%.5f" % IL3_drop3, "%.3f" % coverage3_drop3)

0.00440 0.00109 0.24745 0.933
0.00516 -0.00577 0.27672 0.945
0.00671 -0.00119 0.34228 0.955


### Architecture 4

In [200]:
se4_drop, ate4_drop, IL4_drop, coverage4_drop, tautruemean4_drop =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10,5],estimate_ps=False,dropout_rates=[.20,.20,.20,.20])

In [201]:
se4_drop2, ate4_drop2, IL4_drop2, coverage4_drop2, tautruemean4_drop2 =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10,5],estimate_ps=False,dropout_rates=[.40,.40,.40,.40])

In [202]:
se4_drop3, ate4_drop3, IL4_drop3, coverage4_drop3, tautruemean4_drop3 =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10,5],estimate_ps=False,dropout_rates=[.60,.60,.60,.60])

In [203]:
mse4_drop, bias4_drop, IL4_drop, coverage4_drop=calculate_statistics(se4_drop, tautruemean4_drop, ate4_drop, IL4_drop, coverage4_drop)
mse4_drop2, bias4_drop2, IL4_drop2, coverage4_drop2=calculate_statistics(se4_drop2, tautruemean4_drop2, ate4_drop2, IL4_drop2, coverage4_drop2)
mse4_drop3, bias4_drop3, IL4_drop3, coverage4_drop3=calculate_statistics(se4_drop3, tautruemean4_drop3, ate4_drop3, IL4_drop3, coverage4_drop3)

In [204]:
print(mse4_drop, bias4_drop, IL4_drop, coverage4_drop)
print(mse4_drop2, bias4_drop2, IL4_drop2, coverage4_drop2)
print(mse4_drop3, bias4_drop3, IL4_drop3, coverage4_drop3)

0.006074987206304096 -0.00786720720868228 0.30584717094552466 0.948
0.006016010608985462 -0.004251143780506839 0.3151750600042618 0.956
0.007195885566607402 -0.0025131630063728316 0.37120268453913685 0.957


### Architecture 5

In [205]:
se5_drop, ate5_drop, IL5_drop, coverage5_drop, tautruemean5_drop =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10,5],estimate_ps=False,dropout_rates=[.20,.20,.20,.20])

In [206]:
se5_drop2, ate5_drop2, IL5_drop2, coverage5_drop2, tautruemean5_drop2 =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10,5],estimate_ps=False,dropout_rates=[.40,.40,.40,.40])

In [207]:
se5_drop3, ate5_drop3, IL5_drop3, coverage5_drop3, tautruemean5_drop3 =simulator(False, N=1000, runs=1000, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10,5],estimate_ps=False,dropout_rates=[.60,.60,.60,.60])

In [209]:
mse5_drop, bias5_drop, IL5_drop, coverage5_drop=calculate_statistics(se5_drop, tautruemean5_drop, ate5_drop, IL5_drop, coverage5_drop)
mse5_drop2, bias5_drop2, IL5_drop2, coverage5_drop2=calculate_statistics(se5_drop2, tautruemean5_drop2, ate5_drop2, IL5_drop2, coverage5_drop2)
mse5_drop3, bias5_drop3, IL5_drop3, coverage5_drop3=calculate_statistics(se5_drop3, tautruemean53_drop3, ate5_drop3, IL5_drop3, coverage5_drop3)

In [210]:
print(mse5_drop, bias5_drop, IL5_drop, coverage5_drop)
print(mse5_drop2, bias5_drop2, IL5_drop2, coverage5_drop2)
print(mse5_drop3, bias5_drop3, IL5_drop3, coverage5_drop3)

0.006061279640277237 -0.006917304038295491 0.3041019367581171 0.946
0.005698496775378494 -0.006756835011470175 0.313142442818595 0.959
0.006692293954184345 -0.005326453807319886 0.3705088948670396 0.963


## Convergence properties
#### Strategy is to examine behavior of different structures at consecutively higher sample sizes
##### Architecture 1

In [3]:
se1_con, ate1_con, IL1_con, coverage1_con, tautruemean1_con =simulator(False, N=100, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10],estimate_ps=False,dropout_rates=None)

In [4]:
se1_con2, ate1_con2, IL1_con2, coverage1_con2, tautruemean1_con2 =simulator(False, N=250, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10],estimate_ps=False,dropout_rates=None)

In [5]:
se1_con3, ate1_con3, IL1_con3, coverage1_con3, tautruemean1_con3 =simulator(False, N=500, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10],estimate_ps=False,dropout_rates=None)

In [6]:
se1_con4, ate1_con4, IL1_con4, coverage1_con4, tautruemean1_con4 =simulator(False, N=750, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10],estimate_ps=False,dropout_rates=None)

In [7]:
se1_con5, ate1_con5, IL1_con5, coverage1_con5, tautruemean1_con5 =simulator(False, N=1000, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10],estimate_ps=False,dropout_rates=None)

In [8]:
se1_con6, ate1_con6, IL1_con6, coverage1_con6, tautruemean1_con6 =simulator(False, N=2000, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10],estimate_ps=False,dropout_rates=None)

In [9]:
se1_con7, ate1_con7, IL1_con7, coverage1_con7, tautruemean1_con7 =simulator(False, N=5000, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10],estimate_ps=False,dropout_rates=None)

In [10]:
mse1_con, bias1_con, IL1_con, coverage1_con=calculate_statistics(se1_con, tautruemean1_con, ate1_con, IL1_con, coverage1_con)
mse1_con2, bias1_con2, IL1_con2, coverage1_con2=calculate_statistics(se1_con2, tautruemean1_con2, ate1_con2, IL1_con2, coverage1_con2)
mse1_con3, bias1_con3, IL1_con3, coverage1_con3=calculate_statistics(se1_con3, tautruemean1_con3, ate1_con3, IL1_con3, coverage1_con3)
mse1_con4, bias1_con4, IL1_con4, coverage1_con4=calculate_statistics(se1_con4, tautruemean1_con4, ate1_con4, IL1_con4, coverage1_con4)
mse1_con5, bias1_con5, IL1_con5, coverage1_con5=calculate_statistics(se1_con5, tautruemean1_con5, ate1_con5, IL1_con5, coverage1_con5)
mse1_con6, bias1_con6, IL1_con6, coverage1_con6=calculate_statistics(se1_con6, tautruemean1_con6, ate1_con6, IL1_con6, coverage1_con6)
mse1_con7, bias1_con7, IL1_con7, coverage1_con7=calculate_statistics(se1_con7, tautruemean1_con7, ate1_con7, IL1_con7, coverage1_con7)


In [11]:
print("%.5f" % mse1_con, "%.5f" % bias1_con, "%.5f" % IL1_con, "%.3f" % coverage1_con)
print("%.5f" % mse1_con2, "%.5f" % bias1_con2, "%.5f" % IL1_con2, "%.3f" % coverage1_con2)
print("%.5f" % mse1_con3, "%.5f" % bias1_con3, "%.5f" % IL1_con3, "%.3f" % coverage1_con3)
print("%.5f" % mse1_con4, "%.5f" % bias1_con4, "%.5f" % IL1_con4, "%.3f" % coverage1_con4)
print("%.5f" % mse1_con5, "%.5f" % bias1_con5, "%.5f" % IL1_con5, "%.3f" % coverage1_con5)
print("%.5f" % mse1_con6, "%.5f" % bias1_con6, "%.5f" % IL1_con6, "%.3f" % coverage1_con6)
print("%.5f" % mse1_con7, "%.5f" % bias1_con7, "%.5f" % IL1_con7, "%.3f" % coverage1_con7)

0.06979 -0.01957 0.91301 0.900
0.02359 -0.00168 0.52988 0.900
0.00895 -0.00000 0.36325 0.946
0.00597 -0.00615 0.28866 0.934
0.00446 0.00049 0.24891 0.942
0.00186 -0.00311 0.17494 0.958
0.00085 0.00128 0.11105 0.940


##### Architecture 2

In [44]:
se2_con, ate2_con, IL2_con, coverage2_con, tautruemean2_con =simulator(False, N=100, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20],estimate_ps=False,dropout_rates=None)

In [45]:
se2_con2, ate2_con2, IL2_con2, coverage2_con2, tautruemean2_con2 =simulator(False, N=250, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20],estimate_ps=False,dropout_rates=None)

In [46]:
se2_con3, ate2_con3, IL2_con3, coverage2_con3, tautruemean2_con3 =simulator(False, N=500, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20],estimate_ps=False,dropout_rates=None)

In [47]:
se2_con4, ate2_con4, IL2_con4, coverage2_con4, tautruemean2_con4 =simulator(False, N=750, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20],estimate_ps=False,dropout_rates=None)

In [48]:
se2_con5, ate2_con5, IL2_con5, coverage2_con5, tautruemean2_con5 =simulator(False, N=1000, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20],estimate_ps=False,dropout_rates=None)

In [49]:
se2_con6, ate2_con6, IL2_con6, coverage2_con6, tautruemean2_con6 =simulator(False, N=2000, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20],estimate_ps=False,dropout_rates=None)

In [50]:
se2_con7, ate2_con7, IL2_con7, coverage2_con7, tautruemean2_con7 =simulator(False, N=5000, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20],estimate_ps=False,dropout_rates=None)

In [51]:
mse2_con, bias2_con, IL2_con, coverage2_con=calculate_statistics(se2_con, tautruemean2_con, ate2_con, IL2_con, coverage2_con)
mse2_con2, bias2_con2, IL2_con2, coverage2_con2=calculate_statistics(se2_con2, tautruemean2_con2, ate2_con2, IL2_con2, coverage2_con2)
mse2_con3, bias2_con3, IL2_con3, coverage2_con3=calculate_statistics(se2_con3, tautruemean2_con3, ate2_con3, IL2_con3, coverage2_con3)
mse2_con4, bias2_con4, IL2_con4, coverage2_con4=calculate_statistics(se2_con4, tautruemean2_con4, ate2_con4, IL2_con4, coverage2_con4)
mse2_con5, bias2_con5, IL2_con5, coverage2_con5=calculate_statistics(se2_con5, tautruemean2_con5, ate2_con5, IL2_con5, coverage2_con5)
mse2_con6, bias2_con6, IL2_con6, coverage2_con6=calculate_statistics(se2_con6, tautruemean2_con6, ate2_con6, IL2_con6, coverage2_con6)
mse2_con7, bias2_con7, IL2_con7, coverage2_con7=calculate_statistics(se2_con7, tautruemean2_con7, ate2_con7, IL2_con7, coverage2_con7)

In [52]:
print("%.5f" % mse2_con, "%.5f" % bias2_con, "%.5f" % IL2_con, "%.3f" % coverage2_con)
print("%.5f" % mse2_con2, "%.5f" % bias2_con2, "%.5f" % IL2_con2, "%.3f" % coverage2_con2)
print("%.5f" % mse2_con3, "%.5f" % bias2_con3, "%.5f" % IL2_con3, "%.3f" % coverage2_con3)
print("%.5f" % mse2_con4, "%.5f" % bias2_con4, "%.5f" % IL2_con4, "%.3f" % coverage2_con4)
print("%.5f" % mse2_con5, "%.5f" % bias2_con5, "%.5f" % IL2_con5, "%.3f" % coverage2_con5)
print("%.5f" % mse2_con6, "%.5f" % bias2_con6, "%.5f" % IL2_con6, "%.3f" % coverage2_con6)
print("%.5f" % mse2_con7, "%.5f" % bias2_con7, "%.5f" % IL2_con7, "%.3f" % coverage2_con7)

0.05537 -0.01955 0.82487 0.898
0.01836 -0.01067 0.49962 0.936
0.00938 -0.00177 0.34301 0.920
0.00659 -0.00049 0.28110 0.922
0.00461 -0.00255 0.24356 0.930
0.00206 -0.00253 0.17395 0.948
0.00088 -0.00185 0.11082 0.934


##### Architecture 3

In [54]:
se3_con, ate3_con, IL3_con, coverage3_con, tautruemean3_con =simulator(False, N=100, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[80,80,80],estimate_ps=False,dropout_rates=None)

In [55]:
se3_con2, ate3_con2, IL3_con2, coverage3_con2, tautruemean3_con2 =simulator(False, N=250, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[80,80,80],estimate_ps=False,dropout_rates=None)

In [56]:
se3_con3, ate3_con3, IL3_con3, coverage3_con3, tautruemean3_con3 =simulator(False, N=500, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[80,80,80],estimate_ps=False,dropout_rates=None)

In [57]:
se3_con4, ate3_con4, IL3_con4, coverage3_con4, tautruemean3_con4 =simulator(False, N=750, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[80,80,80],estimate_ps=False,dropout_rates=None)

In [58]:
se3_con5, ate3_con5, IL3_con5, coverage3_con5, tautruemean3_con5 =simulator(False, N=1000, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[80,80,80],estimate_ps=False,dropout_rates=None)

In [59]:
se3_con6, ate3_con6, IL3_con6, coverage3_con6, tautruemean3_con6 =simulator(False, N=2000, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[80,80,80],estimate_ps=False,dropout_rates=None)

In [60]:
se3_con7, ate3_con7, IL3_con7, coverage3_con7, tautruemean3_con7 =simulator(False, N=5000, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[80,80,80],estimate_ps=False,dropout_rates=None)

In [61]:
mse3_con, bias3_con, IL3_con, coverage3_con=calculate_statistics(se3_con, tautruemean3_con, ate3_con, IL3_con, coverage3_con)
mse3_con2, bias3_con2, IL3_con2, coverage3_con2=calculate_statistics(se3_con2, tautruemean3_con2, ate3_con2, IL3_con2, coverage3_con2)
mse3_con3, bias3_con3, IL3_con3, coverage3_con3=calculate_statistics(se3_con3, tautruemean3_con3, ate3_con3, IL3_con3, coverage3_con3)
mse3_con4, bias3_con4, IL3_con4, coverage3_con4=calculate_statistics(se3_con4, tautruemean3_con4, ate3_con4, IL3_con4, coverage3_con4)
mse3_con5, bias3_con5, IL3_con5, coverage3_con5=calculate_statistics(se3_con5, tautruemean3_con5, ate3_con5, IL3_con5, coverage3_con5)
mse3_con6, bias3_con6, IL3_con6, coverage3_con6=calculate_statistics(se3_con6, tautruemean3_con6, ate3_con6, IL3_con6, coverage3_con6)
mse3_con7, bias3_con7, IL3_con7, coverage3_con7=calculate_statistics(se3_con7, tautruemean3_con7, ate3_con7, IL3_con7, coverage3_con7)

In [62]:
print("%.5f" % mse3_con, "%.5f" % bias3_con, "%.5f" % IL3_con, "%.3f" % coverage3_con)
print("%.5f" % mse3_con2, "%.5f" % bias3_con2, "%.5f" % IL3_con2, "%.3f" % coverage3_con2)
print("%.5f" % mse3_con3, "%.5f" % bias3_con3, "%.5f" % IL3_con3, "%.3f" % coverage3_con3)
print("%.5f" % mse3_con4, "%.5f" % bias3_con4, "%.5f" % IL3_con4, "%.3f" % coverage3_con4)
print("%.5f" % mse3_con5, "%.5f" % bias3_con5, "%.5f" % IL3_con5, "%.3f" % coverage3_con5)
print("%.5f" % mse3_con6, "%.5f" % bias3_con6, "%.5f" % IL3_con6, "%.3f" % coverage3_con6)
print("%.5f" % mse3_con7, "%.5f" % bias3_con7, "%.5f" % IL3_con7, "%.3f" % coverage3_con7)

0.06163 -0.00124 0.79456 0.852
0.01915 -0.00280 0.48561 0.902
0.00890 0.00607 0.33803 0.926
0.00533 -0.00045 0.27796 0.950
0.00456 0.00207 0.24275 0.926
0.00214 -0.00189 0.17351 0.950
0.00085 0.00030 0.11082 0.950


##### Architecture 4

In [65]:
se4_con, ate4_con, IL4_con, coverage4_con, tautruemean4_con =simulator(False, N=100, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10,5],estimate_ps=False,dropout_rates=None)

In [66]:
se4_con2, ate4_con2, IL4_con2, coverage4_con2, tautruemean4_con2 =simulator(False, N=250, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10,5],estimate_ps=False,dropout_rates=None)

In [67]:
se4_con3, ate4_con3, IL4_con3, coverage4_con3, tautruemean4_con3 =simulator(False, N=500, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10,5],estimate_ps=False,dropout_rates=None)

In [68]:
se4_con4, ate4_con4, IL4_con4, coverage4_con4, tautruemean4_con4 =simulator(False, N=750, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10,5],estimate_ps=False,dropout_rates=None)

In [69]:
se4_con5, ate4_con5, IL4_con5, coverage4_con5, tautruemean4_con5 =simulator(False, N=1000, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10,5],estimate_ps=False,dropout_rates=None)

In [70]:
se4_con6, ate4_con6, IL4_con6, coverage4_con6, tautruemean4_con6 =simulator(False, N=2000, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10,5],estimate_ps=False,dropout_rates=None)

In [71]:
se4_con7, ate4_con7, IL4_con7, coverage4_con7, tautruemean4_con7 =simulator(False, N=5000, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[20,15,10,5],estimate_ps=False,dropout_rates=None)

In [72]:
mse4_con, bias4_con, IL4_con, coverage4_con=calculate_statistics(se4_con, tautruemean4_con, ate4_con, IL4_con, coverage4_con)
mse4_con2, bias4_con2, IL4_con2, coverage4_con2=calculate_statistics(se4_con2, tautruemean4_con2, ate4_con2, IL4_con2, coverage4_con2)
mse4_con3, bias4_con3, IL4_con3, coverage4_con3=calculate_statistics(se4_con3, tautruemean4_con3, ate4_con3, IL4_con3, coverage4_con3)
mse4_con4, bias4_con4, IL4_con4, coverage4_con4=calculate_statistics(se4_con4, tautruemean4_con4, ate4_con4, IL4_con4, coverage4_con4)
mse4_con5, bias4_con5, IL4_con5, coverage4_con5=calculate_statistics(se4_con5, tautruemean4_con5, ate4_con5, IL4_con5, coverage4_con5)
mse4_con6, bias4_con6, IL4_con6, coverage4_con6=calculate_statistics(se4_con6, tautruemean4_con6, ate4_con6, IL4_con6, coverage4_con6)
mse4_con7, bias4_con7, IL4_con7, coverage4_con7=calculate_statistics(se4_con7, tautruemean4_con7, ate4_con7, IL4_con7, coverage4_con7)

In [73]:
print("%.5f" % mse4_con, "%.5f" % bias4_con, "%.5f" % IL4_con, "%.4f" % coverage4_con)
print("%.5f" % mse4_con2, "%.5f" % bias4_con2, "%.5f" % IL4_con2, "%.4f" % coverage4_con2)
print("%.5f" % mse4_con3, "%.5f" % bias4_con3, "%.5f" % IL4_con3, "%.4f" % coverage4_con3)
print("%.5f" % mse4_con4, "%.5f" % bias4_con4, "%.5f" % IL4_con4, "%.4f" % coverage4_con4)
print("%.5f" % mse4_con5, "%.5f" % bias4_con5, "%.5f" % IL4_con5, "%.4f" % coverage4_con5)
print("%.5f" % mse4_con6, "%.5f" % bias4_con6, "%.5f" % IL4_con6, "%.4f" % coverage4_con6)
print("%.5f" % mse4_con7, "%.5f" % bias4_con7, "%.5f" % IL4_con7, "%.4f" % coverage4_con7)

0.08105 -0.03274 0.91234 0.8840
0.03056 -0.03056 0.53273 0.8820
0.01160 -0.01462 0.35975 0.9120
0.00932 -0.02018 0.28837 0.8860
0.00597 -0.01328 0.24925 0.9120
0.00276 -0.00667 0.17542 0.9180
0.00081 -0.00040 0.11141 0.9440


##### Architecture 5

In [74]:
se5_con, ate5_con, IL5_con, coverage5_con, tautruemean5_con =simulator(False, N=100, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20,10],estimate_ps=False,dropout_rates=None)

In [75]:
se5_con2, ate5_con2, IL5_con2, coverage5_con2, tautruemean5_con2 =simulator(False, N=250, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20,10],estimate_ps=False,dropout_rates=None)

In [76]:
se5_con3, ate5_con3, IL5_con3, coverage5_con3, tautruemean5_con3 =simulator(False, N=500, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20,10],estimate_ps=False,dropout_rates=None)

In [82]:
se5_con4, ate5_con4, IL5_con4, coverage5_con4, tautruemean5_con4 =simulator(False, N=750, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20,10],estimate_ps=False,dropout_rates=None)

In [78]:
se5_con5, ate5_con5, IL5_con5, coverage5_con5, tautruemean5_con5 =simulator(False, N=1000, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20,10],estimate_ps=False,dropout_rates=None)

In [79]:
se5_con6, ate5_con6, IL5_con6, coverage5_con6, tautruemean5_con6 =simulator(False, N=2000, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20,10],estimate_ps=False,dropout_rates=None)

In [80]:
se5_con7, ate5_con7, IL5_con7, coverage5_con7, tautruemean5_con7 =simulator(False, N=5000, runs=500, model='linear',sample='randomized', nconsumer_characteristics=20, hidden_layer_sizes=[60,30,20,10],estimate_ps=False,dropout_rates=None)

In [83]:
mse5_con, bias5_con, IL5_con, coverage5_con=calculate_statistics(se5_con, tautruemean5_con, ate5_con, IL5_con, coverage5_con)
mse5_con2, bias5_con2, IL5_con2, coverage5_con2=calculate_statistics(se5_con2, tautruemean5_con2, ate5_con2, IL5_con2, coverage5_con2)
mse5_con3, bias5_con3, IL5_con3, coverage5_con3=calculate_statistics(se5_con3, tautruemean5_con3, ate5_con3, IL5_con3, coverage5_con3)
mse5_con4, bias5_con4, IL5_con4, coverage5_con4=calculate_statistics(se5_con4, tautruemean5_con4, ate5_con4, IL5_con4, coverage5_con4)
mse5_con5, bias5_con5, IL5_con5, coverage5_con5=calculate_statistics(se5_con5, tautruemean5_con5, ate5_con5, IL5_con5, coverage5_con5)
mse5_con6, bias5_con6, IL5_con6, coverage5_con6=calculate_statistics(se5_con6, tautruemean5_con6, ate5_con6, IL5_con6, coverage5_con6)
mse5_con7, bias5_con7, IL5_con7, coverage5_con7=calculate_statistics(se5_con7, tautruemean5_con7, ate5_con7, IL5_con7, coverage5_con7)

In [84]:
print("%.5f" % mse5_con, "%.5f" % bias5_con, "%.5f" % IL5_con, "%.5f" % coverage5_con)
print("%.5f" % mse5_con2, "%.5f" % bias5_con2, "%.5f" % IL5_con2, "%.5f" % coverage5_con2)
print("%.5f" % mse5_con3, "%.5f" % bias5_con3, "%.5f" % IL5_con3, "%.5f" % coverage5_con3)
print("%.5f" % mse5_con4, "%.5f" % bias5_con4, "%.5f" % IL5_con4, "%.5f" % coverage5_con4)
print("%.5f" % mse5_con5, "%.5f" % bias5_con5, "%.5f" % IL5_con5, "%.5f" % coverage5_con5)
print("%.5f" % mse5_con6, "%.5f" % bias5_con6, "%.5f" % IL5_con6, "%.5f" % coverage5_con6)
print("%.5f" % mse5_con7, "%.5f" % bias5_con7, "%.5f" % IL5_con7, "%.5f" % coverage5_con7)

0.05856 -0.01952 0.84556 0.90400
0.02217 -0.02228 0.49468 0.89400
0.00953 -0.01260 0.34322 0.92600
0.00605 -0.00462 0.28062 0.93200
0.00447 -0.00480 0.24361 0.92800
0.00226 -0.00033 0.17351 0.93600
0.00085 -0.00141 0.11069 0.95000
